### 문제 6-1 : 조건부 분기가 있는 메뉴 추천 시스템 ( LangGraph 사용하기)

In [ ]:
import os
from dotenv import load_dotenv
from typing import List, TypedDict, Annotated
import re
import shutil

# LangChain 관련
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

# LangGraph 관련
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages

# LangChain Community 및 기타
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_groq import ChatGroq


# --- 환경 변수 로드 ---
load_dotenv()

# --- 사용자 상태 정의 ---
class CafeState(TypedDict):
    """
    카페 챗봇의 상태를 정의합니다.
    'messages' 필드는 add_messages를 통해 LangGraph가 자동으로 관리합니다.
    """
    messages: Annotated[List[BaseMessage], add_messages]
    question_type: str | None = None  # 분류된 질문 유형


# --- 벡터 DB 생성 및 로드 ---
DATA_DIR = "./data"
DB_PATH = "./db/cafe_db"
MENU_DATA_FILE = os.path.join(DATA_DIR, "cafe_menu_data.txt")

# 데이터 파일이 없으면 생성
os.makedirs(DATA_DIR, exist_ok=True)
if not os.path.exists(MENU_DATA_FILE):
    print(f"⚠️ '{MENU_DATA_FILE}' 파일이 없어 예시 데이터를 생성합니다.")
    with open(MENU_DATA_FILE, "w", encoding="utf-8") as f:
        f.write("아메리카노: 신선한 에스프레소에 물을 더한 클래식 커피. ₩4,000\n")
        f.write("카페 라떼: 부드러운 우유와 에스프레소의 조화. ₩4,500\n")
        f.write("바닐라 라떼: 달콤한 바닐라 시럽이 들어간 라떼. ₩5,000\n")
        f.write("카푸치노: 풍성한 우유 거품이 올라간 커피. ₩4,500\n")
        f.write("초콜릿 라떼: 진한 초콜릿과 우유가 만난 음료. ₩5,500\n")
        f.write("딸기 스무디: 신선한 딸기로 만든 상큼한 스무디. ₩6,000\n")
        f.write("블루베리 베이글: 크림치즈와 잘 어울리는 블루베리 베이글. ₩3,500\n")
        f.write("치즈 케이크: 부드럽고 진한 치즈의 맛. ₩6,500\n")
        f.write("에그 샌드위치: 신선한 재료로 만든 든든한 샌드위치. ₩5,800\n")
    
    if os.path.exists(DB_PATH):
        shutil.rmtree(DB_PATH)
    print("✅ 예시 데이터 파일이 생성되었습니다. 벡터 DB를 새로 생성하려면 스크립트를 다시 실행해주세요.")
    exit()

# 벡터 DB가 없으면 생성하고, 있으면 로드
if not os.path.exists(DB_PATH):
    print("✅ 벡터 DB가 없어 새로 생성합니다...")
    os.makedirs(DB_PATH, exist_ok=True)
    loader = TextLoader(MENU_DATA_FILE, encoding="utf-8")
    docs = loader.load()
    splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    chunks = splitter.split_documents(docs)
    
    embeddings = OllamaEmbeddings(model="bge-m3")
    db = FAISS.from_documents(chunks, embeddings)
    db.save_local(DB_PATH)
    print("✅ 벡터 DB 생성이 완료되었습니다.")
else:
    print("✅ 벡터 DB가 이미 존재합니다.")

embeddings = OllamaEmbeddings(model="bge-m3")
menu_db = FAISS.load_local(DB_PATH, embeddings, allow_dangerous_deserialization=True)
print(f"Ollama Embeddings 모델 (bge-m3)을 사용하여 벡터 DB를 관리합니다.")

# --- 정보 추출 함수 ---
def extract_menu_info(doc: Document) -> dict:
    """Vector DB 문서에서 구조화된 메뉴 정보 (이름, 가격, 설명)를 추출합니다."""
    content = doc.page_content
    name_match = re.match(r'([^:]+):', content)
    menu_name = name_match.group(1).strip() if name_match else 'Unknown'
    price_match = re.search(r'₩([\d,]+)', content)
    price = price_match.group(0) if price_match else "가격 정보 없음"
    description = content
    if name_match:
        description = description.replace(name_match.group(0), "", 1).strip()
    if price_match:
        description = description.replace(price, "").strip()
    if description.startswith(":"):
        description = description[1:].strip()
    return {"name": menu_name, "price": price, "description": description or "설명 없음"}

# --- LLM 초기화 (Groq 사용) ---
llm = ChatGroq(model_name="llama3-8b-8192", temperature=0.7)
print(f"LLM으로 Groq 모델 ({llm.model_name})을 사용합니다.")

# --- 응답 생성 유틸리티 함수 ---
def generate_llm_response(current_messages: List[BaseMessage], context: str, user_query: str) -> str:
    """주어진 컨텍스트와 사용자 쿼리, 대화 이력을 바탕으로 LLM 응답을 생성합니다."""
    recent_history = current_messages[-5:]
    messages_for_llm = recent_history + [
        HumanMessage(content=f"다음 정보를 바탕으로 고객의 질문에 친절하고 간결하게 답변해주세요. 정보:\n{context}\n\n고객 질문: {user_query}")
    ]
    response = llm.invoke(messages_for_llm)
    return response.content

# --- 노드 함수 정의 ---

def classify_question_node(state: CafeState) -> dict:
    """사용자 문의 유형을 분류하여 state의 question_type을 업데이트합니다."""
    last_message_content = state['messages'][-1].content.lower()
    
    if "가격" in last_message_content or "얼마" in last_message_content:
        q_type = "price"
    elif "추천" in last_message_content or "좋은" in last_message_content or "인기" in last_message_content:
        q_type = "recommend"
    elif any(keyword in last_message_content for keyword in ["메뉴", "디저트", "라떼", "뭐", "어떤", "종류", "있나요"]):
        q_type = "menu"
    else:
        q_type = "general"
    # 상태 업데이트를 위해 딕셔너리를 반환합니다.
    return {"question_type": q_type}

def route_question(state: CafeState) -> str:
    """state에 저장된 question_type에 따라 다음에 실행할 노드 이름을 반환합니다."""
    return state["question_type"]

def answer_price(state: CafeState) -> dict:
    query = state['messages'][-1].content
    docs = menu_db.similarity_search(query + " 가격", k=5)
    items = [extract_menu_info(doc) for doc in docs]
    if items:
        context_str = "\n".join([f"- {i['name']}: {i['price']} ({i['description']})" for i in items])
        response_content = generate_llm_response(state['messages'], context_str, query)
    else:
        response_content = "죄송합니다. 요청하신 메뉴의 가격 정보를 찾을 수 없습니다. 어떤 메뉴의 가격이 궁금하신가요?"
    return {"messages": [AIMessage(content=response_content)]}

def answer_recommend(state: CafeState) -> dict:
    query = state['messages'][-1].content
    docs = menu_db.similarity_search(query, k=3)
    if not docs:
        docs = menu_db.similarity_search("인기 메뉴", k=3)
    items = [extract_menu_info(doc) for doc in docs]
    if items:
        context_str = "\n".join([f"추천 메뉴: {i['name']} - {i['description']} ({i['price']})" for i in items])
        response_content = generate_llm_response(state['messages'], context_str, query)
    else:
        response_content = "지금 추천해 드릴 만한 특별한 메뉴를 찾을 수 없습니다. 어떤 종류의 메뉴를 선호하시나요?"
    return {"messages": [AIMessage(content=response_content)]}

def answer_menu(state: CafeState) -> dict:
    query = state['messages'][-1].content
    docs = menu_db.similarity_search(query, k=4)
    if not docs:
        response_content = "죄송합니다. 해당 메뉴 정보를 찾을 수 없습니다. 메뉴 이름을 정확히 말씀해주시겠어요?"
    else:
        items = [extract_menu_info(doc) for doc in docs]
        context_str = "\n".join([f"{i['name']}: {i['description']} ({i['price']})" for i in items])
        response_content = generate_llm_response(state['messages'], context_str, query)
    return {"messages": [AIMessage(content=response_content)]}

def fallback_answer(state: CafeState) -> dict:
    query = state['messages'][-1].content
    response_content = generate_llm_response(state['messages'], "특별한 메뉴 정보 없음", query)
    return {"messages": [AIMessage(content=response_content)]}


# --- LangGraph 그래프 구성 (수정된 부분) ---
graph = StateGraph(CafeState)

# 각 노드를 문자열 이름과 함께 추가합니다.
graph.add_node("classify", classify_question_node)
graph.add_node("price", answer_price)
graph.add_node("recommend", answer_recommend)
graph.add_node("menu", answer_menu)
graph.add_node("general", fallback_answer)

# 진입점을 'classify' 노드로 설정합니다.
graph.set_entry_point("classify")

# 'classify' 노드에서 시작하는 조건부 엣지를 추가합니다.
graph.add_conditional_edges(
    source="classify",
    path=route_question,
    path_map={
        "price": "price",
        "recommend": "recommend",
        "menu": "menu",
        "general": "general",
    }
)

# 각 응답 노드는 대화를 종료(END)합니다.
graph.add_edge("price", END)
graph.add_edge("recommend", END)
graph.add_edge("menu", END)
graph.add_edge("general", END)


# --- 그래프 컴파일 ---
app = graph.compile()

# --- 챗봇 실행 ---
if __name__ == "__main__":
    print("\n--- ☕️ 카페 챗봇에 오신 것을 환영합니다! ---")
    print("메뉴, 가격, 추천 등에 대해 물어보실 수 있습니다. '종료'라고 입력하면 대화가 끝납니다.")
    
    messages = []
    while True:
        user_input = input("\n💁‍♂️ 고객님: ")
        if user_input.lower() == "종료":
            print("👋 챗봇: 다음에 또 만나요!")
            break

        input_message = HumanMessage(content=user_input)
        
        result = app.invoke({"messages": messages + [input_message]})

        ai_response_message = result["messages"][-1]
        print(f"🤖 챗봇: {ai_response_message.content}")
        
        # 다음 대화를 위해 전체 메시지 이력 업데이트
        messages = result["messages"]

✅ 벡터 DB가 이미 존재합니다.
Ollama Embeddings 모델 (bge-m3)을 사용하여 벡터 DB를 관리합니다.


C:\Users\user1\AppData\Local\Temp\ipykernel_16272\2940869297.py:78: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="bge-m3")


LLM으로 Groq 모델 (llama3-8b-8192)을 사용합니다.

--- ☕️ 카페 챗봇에 오신 것을 환영합니다! ---
메뉴, 가격, 추천 등에 대해 물어보실 수 있습니다. '종료'라고 입력하면 대화가 끝납니다.
